# Data Wrangling for NHANES Data

## Summary

In this notebook we will clean data which was collected from the NHANES website: www.cdc.gov/nchs/nhanes/index.htm. This is the first step towards creating a predictive model for hypertension and diabetes. Aside from typical cleaning tasks carried out in data wrangling, there are special considerations due to the nature of the survey.

### Filling in cells skipped by design in the survey

The NHANES survey methods indicate occasionally skipping questions based on previous answers. For example, if the answer to the question 'Have you smoked 100 cigarettes in your lifetime?' is no, then the following question 'Are you currently smoking?' is skipped. In such columns we expect large numbers of missing values and they are easily filled in.

### Treating refused / don't know as missing

The NHANES survey taker records responses of the SP 'refused(to answer)' and 'don't know'. Such answers are coded as numbers which are documented on the NHANES website. There are not enough of these values overall to treat them as a separate category, so we will treat them as we treat the other missing values in the data.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_pickle("raw_data.pkl")

In [3]:
# Before we begin cleaning we drop SPs below age 20 
df = df[df.RIDAGEYR >= 20]
# Change floating points near zero to zero:
df = df.round()

In [4]:
print(f'The size of the dataset: {df.shape}')
# Let us view the data
df.head()

The size of the dataset: (34770, 37)


,DMDHREDU,RIDRETH1,RIAGENDR,RIDAGEYR,INDHHIN2,ALQ150,BPQ020,BPQ080,DID040,DIQ010,...,BMXBMI,BMXWAIST,BMXLEG,BMXARMC,BMXARML,PHAFSTHR,LBXGLU,OHQ845,ALQ151,DMDHREDZ
SEQN,,,,,,,,,,,,,,,,,,,,,
41475.0,4.0,5.0,2.0,62.0,6.0,2.0,1.0,2.0,NaN,2.0,...,58.0,156.0,34.0,45.0,38.0,7.0,NaN,NaN,NaN,NaN
41477.0,3.0,3.0,1.0,71.0,5.0,2.0,1.0,1.0,60.0,1.0,...,30.0,110.0,32.0,34.0,38.0,2.0,NaN,NaN,NaN,NaN
41479.0,1.0,1.0,1.0,52.0,8.0,2.0,2.0,NaN,NaN,2.0,...,28.0,95.0,33.0,33.0,34.0,14.0,113.0,NaN,NaN,NaN
41481.0,4.0,4.0,1.0,21.0,6.0,2.0,2.0,NaN,NaN,2.0,...,23.0,80.0,44.0,31.0,43.0,12.0,NaN,NaN,NaN,NaN
41482.0,4.0,1.0,1.0,64.0,15.0,1.0,1.0,2.0,NaN,2.0,...,34.0,117.0,44.0,33.0,40.0,1.0,NaN,NaN,NaN,NaN


In [5]:
# Let us view the number of missing values in each column:  
print('Details about each column.\n')
df.info()


Details about each column.

<class 'pandas.core.frame.DataFrame'>
Float64Index: 34770 entries, 41475.0 to 102956.0
Data columns (total 37 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DMDHREDU  28300 non-null  float64
 1   RIDRETH1  34770 non-null  float64
 2   RIAGENDR  34770 non-null  float64
 3   RIDAGEYR  34770 non-null  float64
 4   INDHHIN2  33998 non-null  float64
 5   ALQ150    9086 non-null   float64
 6   BPQ020    34770 non-null  float64
 7   BPQ080    31192 non-null  float64
 8   DID040    4624 non-null   float64
 9   DIQ010    34770 non-null  float64
 10  DBQ197    34770 non-null  float64
 11  DBD900    26393 non-null  float64
 12  DBD895    34770 non-null  float64
 13  KIQ026    34769 non-null  float64
 14  KIQ022    34769 non-null  float64
 15  KIQ005    30156 non-null  float64
 16  OHQ011    5935 non-null   float64
 17  PAQ665    34770 non-null  float64
 18  PAQ650    34770 non-null  float64
 19  PAQ605    34770 non-null  fl

### Combining similar columns

The column pairs (DMDREDU,DMDREDZ), (ALQ150,ALQ151), and (OHQ011,OHQ845) are essentially similar questions whose wording / categorization was slightly modified over the survey cycles. We first combine these columns

In [6]:
# Combine ALQ150 and ALQ151
df.loc[df['ALQ150'].isna(), 'ALQ150'] = df['ALQ151']
df = df.drop(['ALQ151'],axis = 1)

# Combine DMDHREDU and DMDHREDZ
# DMDHREDU must be recoded before combination:
df.loc[(df['DMDHREDU'] == 2),'DMDHREDU'] = 1
df.loc[(df['DMDHREDU'] == 3) | (df['DMDHREDU'] == 4),'DMDHREDU'] = 2
df.loc[(df['DMDHREDU'] == 5),'DMDHREDU'] = 3
# Combination:
df.loc[df['DMDHREDU'].isna(), 'DMDHREDU'] = df['DMDHREDZ']
df = df.drop(['DMDHREDZ'],axis = 1)

# Combine OHQ011 and OHQ845
# OHQ011 must be recoded before combination
df['OHQ011'] = df['OHQ011'] - 10
# Combination:
df.loc[df['OHQ011'].isna(), 'OHQ011'] = df['OHQ845']
df = df.drop(['OHQ845'],axis = 1)


### Fixing dependent columns

Again, we note some columns are missing a significant number of values. Most of these are due to the survey methodology of skipping certain questions based on previous answers, we will fill in these values first. 

In [7]:

# If SP has not been told they have diabetes, then no age value.
# First we will simplify the diabetes column
# DIQ010 (refused / don't know/ missing) --> No
df.loc[(df['DIQ010'] == 7) | (df['DIQ010'] == 9),'DIQ010'] = np.nan
df.loc[df.DIQ010.isna(),'DIQ010'] = 2 
# Simplify diabetes age column
# replace < 1 yr w 1 and (refused / don't know) --> median
df.loc[(df['DID040'] == 666),'DID040'] = 1  
df.loc[(df['DID040'] == 777) | (df['DID040'] == 999) ,'DID040'] = df['DID040'].median() 
# Code <= 40 or > 40
df.loc[(df['DID040'] <= 40),'DID040'] = 1 
df.loc[(df['DID040'] > 40),'DID040'] = 3  
# For those told they have borderline case add new code 
df.loc[(df['DIQ010'] == 3),'DID040'] = 2  
# For those not told they have diabetes, add new code
df.loc[(df['DIQ010'] == 2),'DID040'] = 0   

# For the Fast food we will fill in 0 if no meals were eaten out
df.loc[(df['DBD895'] < .5),'DBD900'] = 0    

# If SP has not smoked 100 cigarettes, then not currently smoking
df.loc[(df['SMQ020'] > 1),'SMQ040'] = 3    

# We need the target values: the blood preasure measurements, so we drop those rows missing these values
df = df.dropna(subset = ['BPXSY1','BPXDI1'])

# Now let us view missing values again
print('Number of missing values per column after filling in skipped questions and dropping rows missing blood pressure.')
df.isna().sum(axis=0)

Number of missing values per column after filling in skipped questions and dropping rows missing blood pressure.


DMDHREDU      922
RIDRETH1        0
RIAGENDR        0
RIDAGEYR        0
INDHHIN2      546
ALQ150       6372
BPQ020          0
BPQ080       3180
DID040          3
DIQ010          0
DBQ197          0
DBD900         23
DBD895          0
KIQ026          1
KIQ022          1
KIQ005       2429
OHQ011       1244
PAQ665          0
PAQ650          0
PAQ605          0
PAQ620          0
PAQ635          0
SMQ040          1
SMQ020          1
BPXSY1          0
BPXDI1          0
BPXPLS          2
BMXBMI        340
BMXWAIST     1415
BMXLEG       1470
BMXARMC      1102
BMXARML      1097
PHAFSTHR      473
LBXGLU      16526
dtype: int64

Note there are still a large number of missing values for LBXGLU, this is blood sugar which will be used to define our target variable of being (pre)diabetic, we will drop the rows containing missing values only when we specialize to the predictive diabetes model.

In [8]:
# Viewing the number of missing values per row we find the majority have 2 or fewer
print('View the number of rows missing k values:')
df.isna().sum(axis=1).value_counts().sort_index()

View the number of rows missing k values:


0      9152
1     13233
2      4877
3      1819
4       481
5       335
6       287
7       282
8       121
9        40
10        7
11        1
dtype: int64

## Recoding / Cleaning

We now run through each survey cleaning and recoding the columns.

### Demographics

The demographics survey includes the following features
  * Age
  * Gender
  * Ethnicity
  * Education
  * Household Income
 

In [9]:
# We will clean each survey one at a time, beginning with the Demographics survey.

# Demographics
# RIDAGEYR age  
# RIAGENDR gender OK
# RIDRETH1 ethnicity OK
 

df = df.rename(columns={'RIAGENDR':'Gender','RIDAGEYR':'Age','RIDRETH1':'Ethnicity'})

### Education and Income

Note above there are missing values in the education DMDREDU and Income INDHHIN2 columns. We will fill in the missing education values with the mode. The missing income values will be filled with the mean income of the their eduacation level.

In [10]:
# Education

# Fill in Don't know/ Refused/ Missing --> Mode 
df.loc[ (df['DMDHREDU'] == 7) | (df['DMDHREDU'] == 9), 'DMDHREDU'] = np.nan
df.loc[df['DMDHREDU'].isna(), 'DMDHREDU'] = df['DMDHREDU'].mode()[0]

# Household Income 
# Under 20K
df.loc[  df['INDHHIN2'].isin([1,2,3,4,12]), 'INDHHIN2'] = 1
# 20K to 45K
df.loc[  df['INDHHIN2'].isin([5,6,7]), 'INDHHIN2'] = 2
# 45K to 75K
df.loc[  df['INDHHIN2'].isin([8,9,10]), 'INDHHIN2'] = 3
# Over 75K
df.loc[  df['INDHHIN2'].isin([14,15]), 'INDHHIN2'] = 4
# Fill in Don't know/ Refused/ Over 20K/  Missing --> mode per education group
df.loc[  df['INDHHIN2'].isin([13,77,99]), 'INDHHIN2'] = np.nan

# Impute most common income per education level:
edu_inc = df.groupby(by=["DMDHREDU"])["INDHHIN2"].agg(pd.Series.mode).to_dict()
def edu_inc_impute(a,b):
    if np.isnan(b):
        return edu_inc[a]
    else:
        return b
df.loc[df['INDHHIN2'].isna(),'INDHHIN2'] = df.apply(lambda x: edu_inc_impute(x.DMDHREDU,x.INDHHIN2) ,axis = 1)

# Rename columns

df = df.rename(columns={'INDHHIN2':'HHIncome','DMDHREDU':'Education'})


### Alcohol

Ever have 4/5 or more drinks every day?
 

In [11]:
#Alcohol
df.loc[ (df['ALQ150'] == 7) | (df['ALQ150'] == 9), 'ALQ150'] = np.nan
df.loc[df['ALQ150'].isna(), 'ALQ150'] = 2

df = df.rename(columns={'ALQ150':'Alcohol'})

### Hypertension / Cholesterol

Have you been told by your doctor you have hypertension / high cholesterol?

In [12]:
# Blood Pressure & Cholesterol

# Told you have Hypertension -- refused / don't know / missing --> No 
df.loc[ (df['BPQ020'] == 7) | (df['BPQ020'] == 9), 'BPQ020'] = np.nan
df.loc[df['BPQ020'].isna(), 'BPQ020'] = 2
    
# Told High Cholestorol -- refused / don't know / missing --> No 
df.loc[ (df['BPQ080'] == 7) | (df['BPQ080'] == 9), 'BPQ080'] = np.nan
df.loc[df['BPQ080'].isna(), 'BPQ080'] = 2 

df = df.rename(columns={'BPQ020':'HT_YN','BPQ080':'CHOL_YN'})

### Diabetes

The diabetes column has already been partially cleaned above, here we simply fill in missing values and drop the redundant column.

In [13]:
# Diabetes

# Fill in missing value in DID040 with code 0 -- not told have diabetes
df.loc[df.DID040.isna(), 'DID040'] = 0  
# DIQ010 is now redundant, we drop this column 
df = df.drop(['DIQ010'],axis = 1)

df = df.rename(columns={'DID040':'Diabetes'})

In [14]:
# Diet Behavior & Nutrition

# Past 30 days milk consumption.
# 'DBQ197' Milk consumption
# Default = Sometimes
df.loc[(df.DBQ197 > 3), 'DBQ197'] = 2 
df.loc[(df.DBQ197.isna()), 'DBQ197'] = 2  

# How many meals out of the home?
# DBD895 > 21 meals -> 22 meals
df.loc[(df.DBD895 == 5555), 'DBD895'] = 22
# replace (refused / don't know/ missing ) --> median value
df.loc[(df.DBD895 == 7777) | (df.DBD895 == 9999), 'DBD895'] = np.nan
df.loc[df.DBD895.isna(), 'DBD895'] = df.DBD895.median()

# How many fast food meals?
# DBD900 
df.loc[(df.DBD895 == 0), 'DBD900'] = 0 
# DBD900 > 22 meals -> 22 meals
df.loc[(df.DBD900 == 5555), 'DBD900'] = 22  
# replace (refused / don't know / missing) --> median value
df.loc[(df.DBD900 == 7777) | (df.DBD900 == 9999), 'DBD900'] = np.nan
df.loc[df.DBD900.isna(), 'DBD900'] = df.DBD900.median() 

df = df.rename(columns={'DBQ197':'Milk','DBD895':'MealsOut','DBD900':'FastFood'})

In [15]:

# Kidney questionaire
    
# Told Weak kidney (refused / don't know/ missing) --> No
df.loc[(df.KIQ022 == 7) | (df.KIQ022 == 9),'KIQ022'] = np.nan
df.loc[df.KIQ022.isna(),'KIQ022'] = 2 
    
# Kidney stones (refused / don't know/ missing) --> No
df.loc[(df.KIQ026 == 7) | (df.KIQ026 == 9),'KIQ026'] = np.nan
df.loc[df.KIQ026.isna(), 'KIQ026'] = 2  
    
# Urinary leakage (refused / don't know/ missing --> Never
df.loc[(df.KIQ005 == 7) | (df.KIQ005 == 9),'KIQ005'] = np.nan
df.loc[df.KIQ005.isna(),'KIQ005'] = 1

df = df.rename(columns={'KIQ022':'WeakKidneys','KIQ026':'KidneyStones','KIQ005':'UrineLeak'})

### Dental Health

Rate the overall health of your teeth and gums

In [16]:
# Dental health (refused / don't know/ missing) --> good
df.loc[df.OHQ011 > 5,'OHQ011'] = np.nan
df.loc[df.OHQ011.isna(),'OHQ011'] = 3

df = df.rename(columns = {'OHQ011':'Dental'})

### Physical Activity

Does your job involve moderate/vigorous work activity?
Do you walk or bike to work?
Do you participate in moderate/vigorous recreational activity?

In [17]:
# Physical Activity questionaire
    
# Vig work  (refused / don't know/ missing) --> No
df.loc[(df.PAQ605 == 7) | (df.PAQ605 == 9),'PAQ605'] = np.nan
df.loc[df.PAQ605.isna(),'PAQ605'] = 2    
    
# Moderate work  (refused / don't know/ missing) --> No
df.loc[(df.PAQ620 == 7) | (df.PAQ620 == 9),'PAQ620'] = np.nan
df.loc[df.PAQ620.isna(),'PAQ620'] = 2  
    
# Combine moderate and vigorous work
df['Work_Act'] = df[['PAQ605','PAQ620']].min(axis=1)
#df = df.drop(['PAQ605','PAQ620'],axis = 1)

# Walk / Bike
# (refused / don't know / missing) --> No
df.loc[(df.PAQ635 == 7) | (df.PAQ635 == 9),'PAQ635'] = np.nan
df.loc[df.PAQ635.isna(),'PAQ635'] = 2     
        
# Vig rec (refused / don't know/ missing) --> No
df.loc[(df.PAQ650 == 7) | (df.PAQ650 == 9),'PAQ650'] = np.nan
df.loc[df.PAQ650.isna(),'PAQ650'] = 2     
    
# Moderate rec  (refused / don't know/ missing) --> No
df.loc[(df.PAQ665 == 7) | (df.PAQ665 == 9),'PAQ665'] = np.nan
df.loc[df.PAQ665.isna(),'PAQ665'] = 2    
    
# Combine moderate and vigorous rec 
df['Rec_Act'] = df[['PAQ650','PAQ665']].min(axis=1)
#df = df.drop(['PAQ650','PAQ665'],axis = 1)

df = df.rename(columns={'PAQ635':'WalkBike','PAQ605':'Vig_Work','PAQ620':'Mod_Work','PAQ650':'Vig_Rec','PAQ665':'Mod_Rec'})


In [18]:

# Smoking
    
# Smoking 100 (refused / don't know/ missing) --> No
df.loc[(df.SMQ020 == 7) | (df.SMQ020 == 9),'SMQ020'] = np.nan
df.loc[df.SMQ020.isna(),'SMQ020'] = 2   
    
# Smoking 100 = No >> Currently Smoking = No
df.loc[(df.SMQ020 == 2),'SMQ040'] = 3 
    
# Currently Smoking (refused / don't know/ missing) --> No
df.loc[(df.SMQ040 == 7) | (df.SMQ040 == 9),'SMQ040'] = np.nan
df.loc[df.SMQ040.isna(),'SMQ040'] = 3   
    

df = df.rename(columns={'SMQ020':'Smoke_100','SMQ040':'Smoke_now'})

### Body Measurements

Measurements include:
 * BMI
 * Waist
 * Leg Length
 * Arm Length
 * Arm Circumference
 
Missing values will be replaced by the median

In [19]:
# Body measurement

# BMXBMI BMI measurement
# replace missing with median
df.loc[(df.BMXBMI.isna()),'BMXBMI'] = df.BMXBMI.median()
df.loc[(df.BMXWAIST.isna()),'BMXWAIST'] = df.BMXWAIST.median()
df.loc[(df.BMXLEG.isna()),'BMXLEG'] = df.BMXLEG.median()
df.loc[(df.BMXARML.isna()),'BMXARML'] = df.BMXARML.median()
df.loc[(df.BMXARMC.isna()),'BMXARMC'] = df.BMXARMC.median()

df = df.rename(columns={'BMXBMI':'BMI','BMXWAIST':'WAIST','BMXLEG':'LegLen','BMXARML':'ArmLen','BMXARMC':'ArmCirc'})

### Circulatory Measurements

Measurements include:
 * Pulse
 * Systolic pressure
 * Diastolic pressure

Missing pulse values will be replaced with the median. 

Systolic and Diastolic pressure will be used to construct the target variable: Diagnosis of Hypertension.

In [20]:


# Pulse
df.loc[(df.BPXPLS.isna()),'BPXPLS'] = df.BPXPLS.median()

# Let us add a new feature encoding Non-elevated, Elevated, and High blood preasure
df['HT_Diag'] = -1
df.loc[(df.BPXSY1 < 120) & (df.BPXDI1 < 80),'HT_Diag'] = 0 
df.loc[(df.BPXSY1 >= 120) & (df.BPXDI1 < 80),'HT_Diag'] = 1 
df.loc[(df.BPXSY1 >= 130) | (df.BPXDI1 >= 80),'HT_Diag'] = 2  

df = df.rename(columns={'BPXPLS':'Pulse'})

In [21]:
df.HT_Diag.value_counts()

2    13408
0    12176
1     5051
Name: HT_Diag, dtype: int64

In [22]:
# We drop the Systolic and Diastolic measurements
# df = df.drop(columns = ['BPXSY1','BPXDI1'])

In [23]:
# Let's check missing values

df.isna().sum(axis=0)

Education           0
Ethnicity           0
Gender              0
Age                 0
HHIncome            0
Alcohol             0
HT_YN               0
CHOL_YN             0
Diabetes            0
Milk                0
FastFood            0
MealsOut            0
KidneyStones        0
WeakKidneys         0
UrineLeak           0
Dental              0
Mod_Rec             0
Vig_Rec             0
Vig_Work            0
Mod_Work            0
WalkBike            0
Smoke_now           0
Smoke_100           0
BPXSY1              0
BPXDI1              0
Pulse               0
BMI                 0
WAIST               0
LegLen              0
ArmCirc             0
ArmLen              0
PHAFSTHR          473
LBXGLU          16526
Work_Act            0
Rec_Act             0
HT_Diag             0
dtype: int64

In [24]:
df.to_pickle("clean_data.pkl")

In [25]:
df.shape

(30635, 36)